In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import project_lib as mylib
%matplotlib notebook

In [ ]:
thetas = np.linspace(0,90,100) # in degrees
ss = np.linspace(0,1,1000)

f = lambda theta,ss: (theta/90.)*(np.cos(ss*2*np.pi)**2)*(1-ss) # the energy function

f_vals = [f(theta,ss) for theta in thetas] # the values


In [ ]:
x = np.asarray([(1-ss) if theta<= 45 else (1-ss)*np.tan((90-theta)*np.pi/180.) for theta in thetas]) 
y = np.asarray([(1-ss)*np.tan((theta)*np.pi/180.)*(1-ss) if theta <=45 else (1-ss) for theta in thetas])
z = np.asarray(f_vals)

x = np.reshape(x,x.size)
y = np.reshape(y,y.size)
z = np.reshape(z,z.size)

xi = np.linspace(0,1,1000)
yi = np.linspace(0,1,1000)
zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='cubic')

In [ ]:
# contour the gridded data, plotting dots at the randomly spaced data points.
CS = plt.contour(xi,yi,zi,15,linewidths=0.5,colors='k')
CS = plt.contourf(xi,yi,zi,15,cmap=plt.cm.jet)
plt.colorbar() # draw colorbar
# plot data points.
plt.xlim(0,1)
plt.ylim(0,1)
plt.show()

In [ ]:
thetas = range(1,100)
x = np.asarray([(1-ss) if theta<= 45 else (1-ss)*np.tan((90-theta)*np.pi/180.) for theta in thetas]) 
y = np.asarray([(1-ss)*np.tan((theta)*np.pi/180.)*(1-ss) if theta <=45 else (1-ss) for theta in thetas])
energy_levels = mylib.load_object("Nicks_device_square_(radial_energy_levels)(v001)")['energy_levels']
between = (0,1)
z = [energy_levels[theta][:,between[1]]- energy_levels[theta][:,between[0]] for theta in range(0,99)]
z = np.asarray(z)

x = np.reshape(x,x.size)
y = np.reshape(y,y.size)
z = np.reshape(z,z.size)

xi = np.linspace(0,1,1000)
yi = np.linspace(0,1,1000)
zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='cubic')

In [ ]:
# contour the gridded data, plotting dots at the randomly spaced data points.
CS = plt.contour(xi,yi,zi,15,linewidths=0.5,colors='k')
CS = plt.contourf(xi,yi,zi,15,cmap=plt.cm.jet)
plt.colorbar() # draw colorbar
# plot data points.
plt.xlim(0,1)
plt.ylim(0,1)
plt.show()

In [ ]:
energy_levels = mylib.load_object("Nicks_device_square_(radial_energy_levels)(v001)")['energy_levels']
z = [energy_levels[theta][:,between[1]]- energy_levels[theta][:,between[0]] for theta in range(0,99)]
print np.asarray(z).shape

## Accomplish the plotting of the constant energy trace run

In [ ]:
plt.figure()
a_log = np.linspace(0,2,1000)
a_anc = 2-a_log
plt.plot(a_log,a_anc)
plt.show()

In [ ]:
BETWEENLEVELS = (1,3)
DATA = mylib.load_object("Nicks_device_triangle_(radial_energy_levels)(v001)")
a_log_0s = DATA['a_log_0']
ss = np.linspace(0,1,1000)
x = np.asarray([(1-ss)*aL0 for aL0 in a_log_0s]) 
y = np.asarray([(1-ss)*(2-aL0) for aL0 in a_log_0s]) 
energy_levels = DATA['energy_levels']
z = [energy_levels[i][:,BETWEENLEVELS[1]]- energy_levels[i][:,BETWEENLEVELS[0]] for i in range(len(a_log_0s))]
z = np.asarray(z)
p_thetas = np.concatenate([np.asarray([np.pi/2]),np.arctan((2-a_log_0s[1:])/a_log_0s[1:])])

x = np.reshape(x,x.size)
y = np.reshape(y,y.size)
z = np.reshape(z,z.size)

xi = np.linspace(0,2,1000)
yi = np.linspace(0,2,1000)
zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='cubic')

In [ ]:
# contour the gridded data, plotting dots at the randomly spaced data points.
plt.figure()
CS = plt.contour(xi,yi,zi,15,linewidths=0.5,colors='k')
CS = plt.contourf(xi,yi,zi,15,cmap=plt.cm.jet)
plt.colorbar() # draw colorbar
# plot data points.
plt.xlim(0,2)
plt.ylim(0,2)
plt.show()

## Now I should figure out if this changes the trace of the Hamiltonian

In [ ]:
# THIS ALL NEEDS TO BE DEFINED SO THAT WE CAN GENERATE THE PROBLEM AND BASE HAMILTONIANS
points = 1000
#define number of logical and ancilla Qubits
log_qub = 4
anc_qub = log_qub
Qubits = log_qub*2



s = .5 # change the value of s



# define the coupling parameters that comply with the conditions above
JN = 0.1
q0 = 50.
Ja = 100.
Jl = Ja
hl = -Ja+q0

# construct h
hl = np.asarray([-Ja+q0]*log_qub) # h for the logical Qubits
ha = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)]) + np.asarray([JN if (log_qub-i)%2 == 1 else -JN for i in range(1,log_qub+1)])# h for the ancilla Qubits
h_i = np.concatenate((hl,ha)) # the full h

# constuct J
J_log_log = Jl*np.triu(np.ones((log_qub,log_qub)), k=1) # the J component for the logical qubit interactions
J_log_anc = Ja*np.ones((log_qub,log_qub)) # interactions between logical and ancilla bits
Jt = np.concatenate((J_log_log,J_log_anc),axis=1) # make top half of J array
Jb = np.zeros(Jt.shape) # make bottom half of J array
J_ij = np.concatenate((Jt,Jb),axis=0) # the full J

###########
# 'schedules'
sfl = lambda points: np.linspace(0,2,points)*(1-s)
sfa = lambda points: np.linspace(2,0,points)
sfB = lambda points: np.ones(points)*s
##########
sched_funcs = [[sfl]*log_qub+[sfa]*log_qub,[sfB]]
a = mylib.Anneal(Qubits,[h_i,J_ij],T=100.,points = points,sched_funcs = sched_funcs, diff_scheds = True, show_bar = True, light = False) 

H_trace = []
for i in range(points):
    H_trace.append(np.trace(a.H(i)))

plt.figure()
plt.xlabel("$A_{log,0}$ ($2-A_{anc,0}$)")
plt.ylabel("Tr(H)")
plt.title("Does the trace of H stay the same as intended?")
plt.plot(np.linspace(0,2,points), H_trace)
plt.show()

## This seems to work. By choosing the schdules as I have donme I have kept the trace of their Hamiltonians the same and therefore have kept the energy scales the same

In [ ]:
zi[900,900]